In [1]:
from mimikit.freqnet import FreqNet
from mimikit.data import Database, download_database
from mimikit import get_trainer, get_token

In [2]:
# neptune :

neptune_token = get_token()
# for the db :
neptune_project_db = "k-tonal/data-and-base-notebooks"
neptune_exp_id_db = "DAT-1"
full_exp_path_db = neptune_project_db + "/" + neptune_exp_id_db
db_name = "gould.h5"

# for the model :
neptune_project = "k-tonal/model-upload-test"
# those make sense only once you trained your model :
# neptune_exp_id = "EXP-1"
# full_exp_path = neptune_project + "/" + neptune_exp_id


# paths
path_to_db = "./" + db_name
path_to_model = "./model_test"


In [3]:
# get your data

# db = download_database(neptune_token,
#                        full_exp_path_db,
#                        database_name=db_name,
#                        destination="./")
db = Database(path_to_db)

FileNotFoundError: File ./gould.h5 does not exist

In [ ]:
# build a model and its trainer and fit

model = FreqNet(
    # data_object can be an array, a tensor or a db.feature : FreqNet infers its dimensionality!
    data_object=db.fft,
    splits=[.85, .15],
     model_dim=64,
     groups=1,
     n_layers=(2,),
     strict=False,
     accum_outputs=None,
     concat_outputs=None,
     pad_input=None,
     learn_padding=False,
    input_seq_length=32,
                 )

trainer = get_trainer(model=model,
                      max_epochs=2,
                      # epochs controls checkpoints :
                      # if its a int -> checkpoint every x epoch ; if it's a list -> only at [x, y, z, ] epochs
                      epochs=1,
                      # where you will store everything
                      root_dir=path_to_model,
                     neptune_api_token=neptune_token,
                     neptune_project=neptune_project,
                     )

trainer.fit(model)

In [ ]:
for i in range(8):
    audio = model.generate(model.random_train_example(),
                          n_steps=2048,
                          hop_length=db.fft.attrs["hop_length"])
    model.log_audio("test_wav_" + str(i), audio)